In [15]:
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse
from sklearn.naive_bayes import MultinomialNB
import util
from random import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer

def create_data_matrix(direc="train"):
    X = None
    classes = []
    ids = [] 
    i = -1
    for datafile in os.listdir(direc):
        if datafile == '.DS_Store':
            continue

        # extract id and true class (if available) from filename
        id_str, clazz = datafile.split('.')[:2]
        ids.append(id_str)
        # add target class if this is training data
        try:
            classes.append(util.malware_classes.index(clazz))

        except ValueError:
            # we should only fail to find the label in our list of malware classes
            # if this is test data, which always has an "X" label
            assert clazz == "X"
            classes.append(-1)

        # parse file as an xml document
        tree = ET.parse(os.path.join(direc,datafile))
      #  doc_string = call_feats(tree)
      #  if X is None:
      #      X = [doc_string] 
       # else:
       #     X.append(doc_string)

    return np.array(classes), ids

def call_feats(tree):
    doc_string = ''
    doc_string = ''
    for processes in tree.iter():
        for process in processes:
            for thread in process:
                for section in thread:
                    doc_string += section.tag
                    doc_string += ' '
                    if section.tag == 'dump_line':
                        for cmd in section:
                            for details in cmd.items():
                                if details[0] == 'ascii':
                                    doc_string += details[1]
                        doc_string += ' '
                    else:
                        for cmd in section:                       
                            for details in cmd.items():
                                if details[0] != 'filename_hash': 
                                    if details[1].find('&#x') != -1:
                                        doc_string += details[1][ind:ind+5]
                                        doc_string += ' '
                                    
                                    doc_string += '{}_{}_{}'.format(cmd.tag, details[0], details[1])
                                    doc_string += ' '
    return(doc_string)

In [ ]:
X, Y, filenames = create_data_matrix()
X_HOLDOUT, Y_HOLDOUT, filenames_HOLDOUT = create_data_matrix("test")

X_HOLDOUT_FLAT = []
for doc in X_HOLDOUT:
    doc_list = doc.split()
    for word in doc_list:
        X_HOLDOUT_FLAT.append(word)

X_HOLDOUT_FLAT = set(X_HOLDOUT_FLAT)

X_FLAT = []
for doc in X:
    doc_list = doc.split()
    for word in doc_list:
        X_FLAT.append(word)
X_FLAT = set(X_FLAT)

Dict = X_FLAT.intersection(X_HOLDOUT_FLAT)

X_HOLDOUT_Stripped = []
for doc in X_HOLDOUT:
    bag = ''
    doc_list = doc.split()
    for word in doc_list:
        if word in Dict:
            bag += word
            bag += ' '
    X_HOLDOUT_Stripped.append(bag)

X_Stripped = []
for doc in X:
    bag = ''
    doc_list = doc.split()
    for word in doc_list:
        if word in Dict:
            bag += word
            bag += ' '
    X_Stripped.append(bag)

X_Stripped.extend(X_HOLDOUT_Stripped)


In [16]:
Y, filenames = create_data_matrix()

In [ ]:
import pandas as pd
pd.DataFrame(filenames_HOLDOUT).to_csv('IDs.csv')

In [ ]:
pd.DataFrame(filenames_HOLDOUT).to_csv('IDs.csv')

In [ ]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X_Stripped)

In [18]:
msk = np.random.rand(X_Train.shape[0]) < 0.7
#vectorizer = TfidfVectorizer(norm='l2')
#X_tfidf = vectorizer.fit_transform(X_Stripped)
#X_Train = X_tfidf[0:3086]

train = X_Train[msk]
test = X_Train[~msk]
Y1 = Y[msk]
Y2 = Y[~msk]

#X_HOLDOUT = X_tfidf[3086:]

In [21]:
X_Train.shape

(3086, 89623)

In [ ]:
pd.DataFrame(Y).to_csv('Y_Train.csv', index=False)

In [13]:
import pickle
#pickle.dump( X_Train, open( "SparseTrain.p", "wb" ) )
#pickle.dump( X_HOLDOUT, open( "SparseTest.p", "wb" ) )
#pickle.dump( bst, open( "bst.p", "wb" ) )
#bst = pickle.load(open("bst.p", "rb" ) )
X_Train = pickle.load(open("SparseTrain.p", "rb" ) )
X_HOLDOUT = pickle.load(open("SparseTest.p", "rb" ) )
 

In [7]:
bst

In [22]:
#XGBOOST
# convert sparse matrix to DMatrix (data object required for xgBoost)
import xgboost as xgb
#d_train = xgb.DMatrix(X_Train, label = Y)
d_train = xgb.DMatrix(train, label = Y1)
d_test = xgb.DMatrix(test, label = Y2)

# Set parameters 
Conserativeparam = {'objective':'multi:softmax', 'booster':"gbtree", 'eta':.05, 'silent':1,'num_class':15
                    , 'subsample':.5, 'max_depth': 4, 'colsample_bytree':.5, 'colsample_bylevel':.5
                    ,'min_child_weight':1, 'max_delta_step':1, 'gamma':1}

#Defaultparam = {'objective':'multi:softmax', 'booster':"gbtree", 'eta':.05, 'silent':1,'num_class':15}

# Establish watchlist to see model error against test set over boosting rounds
# It's an optional parameter in xgb.train() #

evallist  = [(d_test,'eval'), (d_train,'train')]

# Set number of rounds
num_round = 200

# train model
#bst = xgb.train( Conserativeparam, d_train, num_round, evallist)
bst = xgb.train( Defaultparam, d_train, num_round, evallist)
# predict on set

Y2_Pred = bst.predict(d_test)

[0]	eval-merror:0.110756	train-merror:0.074523
[1]	eval-merror:0.113951	train-merror:0.067070
[2]	eval-merror:0.106496	train-merror:0.061947
[3]	eval-merror:0.104366	train-merror:0.060550
[4]	eval-merror:0.102236	train-merror:0.054495
[5]	eval-merror:0.100106	train-merror:0.052632
[6]	eval-merror:0.100106	train-merror:0.051234
[7]	eval-merror:0.100106	train-merror:0.045645
[8]	eval-merror:0.100106	train-merror:0.044248
[9]	eval-merror:0.100106	train-merror:0.044248
[10]	eval-merror:0.100106	train-merror:0.042850
[11]	eval-merror:0.100106	train-merror:0.040522
[12]	eval-merror:0.101171	train-merror:0.040987
[13]	eval-merror:0.101171	train-merror:0.039590
[14]	eval-merror:0.101171	train-merror:0.039124
[15]	eval-merror:0.101171	train-merror:0.038193
[16]	eval-merror:0.101171	train-merror:0.038193
[17]	eval-merror:0.101171	train-merror:0.038193
[18]	eval-merror:0.102236	train-merror:0.037727
[19]	eval-merror:0.103301	train-merror:0.037261
[20]	eval-merror:0.103301	train-merror:0.036330
[2

In [ ]:
from sklearn.metrics import classification_report
Y2_Pred = bst.predict(d_test)
target_names = ['Agent', 'AutoRun', 'FraudLoad', 'FraudPack', 'Hupigon'
                , 'Krap', 'Lipler', 'Magania', 'None', 'Poison', 'Swizzor'
                , 'Tdss', 'VB', 'Virut', 'Zbot']
print(classification_report(Y2, Y2_Pred, target_names=target_names).ipynb_checkpoints/

In [12]:
#import xgboost as xgb
#import matplotlib
#xgb.plot_tree(bst, num_trees= 2)
xgb.to_graphviz(bst, num_trees=2)

ImportError: You must install graphviz to plot tree

In [ ]:
Preds = list(zip(Y2,Y2_Pred))
sum = 0
for pred in Preds:
    if pred[0]==pred[1]:
        sum+=1
print(sum/len(Preds))

In [ ]:
#Kaggle Submission
d_holdout = xgb.DMatrix(X_HOLDOUT)
Y_Pred_HOLDOUT = bst.predict(d_holdout)
results = list(zip(filenames_HOLDOUT, Y_Pred_HOLDOUT))

In [ ]:
import pandas as pd
Output=pd.DataFrame(results)
Output.to_csv('KaggleSubmission_new5.csv', index=False)